# Custom metadata finder & to-ONNX model converter

**Written by Uglješa Lukešević ([github.com/ukicomputers](https://github.com/ukicomputers)**)

## Preparing packages
1.   Firstly, we need to install an super-gradients library


In [ ]:
!pip install -q super-gradients

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 684.5/684.5 kB 34.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.6/408.6 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 77.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 65.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.0/68.0 kB 8.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel

  2. Copy your model to Colab using file function from left side
  3. Change this three next variables according the comments, and execute that code

In [ ]:
MODEL_TYPE = "yolo_nas_l"  # change this variable to your model type

MODEL_TRAINED_CLASSES = 2  # change this variable to num. of classes that your model is trained)

MODEL_PATH = "./model.pth"  # change this variable to your model path

4. Once you configure that variables, execute the following code, converter and metadata writer

In [ ]:
# Written by Uglješa Lukešević (github.com/ukicomputers)
# TODO: normalizeImage

import numpy as np

std = None
brp = None
cp = None
dr = None
dlmr = None


def get_preprocessing_steps(preprocessing, processing):
    global std, brp, cp, dr, dlmr
    if isinstance(preprocessing, processing.StandardizeImage):
        std = preprocessing.max_value
    elif isinstance(preprocessing, processing.DetectionRescale):
        dr = True
    elif isinstance(preprocessing, processing.DetectionLongestMaxSizeRescale):
        dlmr = True
    elif isinstance(preprocessing, processing.DetectionBottomRightPadding):
        brp = preprocessing.pad_value
    elif isinstance(preprocessing, processing.DetectionCenterPadding):
        cp = preprocessing.pad_value


def main():
    global std, brp, cp, dr, dlmr
    from super_gradients.training import models
    import super_gradients.training.processing as processing

    net = models.get(
        MODEL_TYPE, num_classes=MODEL_TRAINED_CLASSES, checkpoint_path=MODEL_PATH
    )

    dummy = np.random.randint(0, 255, (1000, 800, 3), dtype=np.uint8)

    iou = net._default_nms_iou
    conf = net._default_nms_conf

    for st in net._image_processor.processings:
        get_preprocessing_steps(st, processing)

    imgsz = np.expand_dims(net._image_processor.preprocess_image(dummy)[0], 0).shape

    file = (
        str(iou) + "\n" + str(conf) + "\n" + str(imgsz[2]) + "\n" + str(imgsz[3]) + "\n"
    )

    if std != None:
        file += str(std) + "\n"
    else:
        file += "n" + "\n"

    if dr != None:
        file += "t" + "\n"
    else:
        file += "n" + "\n"

    if dlmr != None:
        file += "t" + "\n"
    else:
        file += "n" + "\n"

    if brp != None:
        file += str(brp) + "\n"
    else:
        file += "n" + "\n"

    if cp != None:
        file += str(cp) + "\n"
    else:
        file += "n"

    """
    metadata output:
        iou thereshold (float)
        score thereshold (float)
        width (int)
        height (int)
        standardize (string t:n, 1:0)
        detect rescale (string t:n, 1:0)
        detect long max rescale (string t:n, 1:0)
        bottom right padding (int:string (if n))
        center padding (int:string (if n))
    """

    filename = f"metadata"
    with open(filename, "w") as f:
        f.write(file)

    models.convert_to_onnx(model=net, input_shape=imgsz[1:], out_path="model.onnx")


if __name__ == "__main__":
    main()


[2023-10-09 17:29:44] INFO - checkpoint_utils.py - Successfully loaded model weights from ./model.pth EMA checkpoint.
[2023-10-09 17:29:44] WARNING - conversion.py - input_shape is deprecated and will be removed in the next major release.Use the convert_to_onnx(..., prep_model_for_conversion_kwargs(input_size=(1, 3, 640, 640))) instead


============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



That's it! From files section, now you can download your **model.onnx** file and your **metadata** file, and that execute it. Happy hacking!